In [1]:
import sys
import os 

WORK_AREA = "../"
os.chdir(WORK_AREA)

paths = ['./src/', './src/api/v6']
for path in paths:
    path = os.path.normcase(path)
    if not any(os.path.normcase(sp) == path for sp in sys.path):
        sys.path.append(path)

In [2]:
import pandas as pd

In [3]:

import ast
import math
from representations.tree.tree import Tree
from representations.builders.ast.tearers.tearer_factory import TearerFactory


def parse_code_rep_to_code(code_rep: str, verbose: str = 'Fatal') -> str:
  try:
    tree = Tree.unparse(code_rep)
    tearer = TearerFactory().get_tearer(tree.root_node)
    asdl = tearer.tear(tree.root_node)
    code = ast.unparse(asdl)
  except Exception as e:
    if verbose == 'Error':
      print(f"[Error] failed to prase code rep to code:\n", e)
    code = ''
  finally:
    return code


def build_test_code(code: str, imports: str, test: str, code_embed_str: str = '# end code block to test', fail_on_error: bool = False, verbose: str = 'Fatal'):
  try:
    code_insert_idx = test.find(code_embed_str)
    program_code = imports
    program_code += '\n'
    program_code += test[:code_insert_idx]
    program_code += code
    program_code += '\n'
    program_code += test[code_insert_idx:]
  except Exception as e:
    if verbose == 'Error':
      print('[ERROR] Failed to unparse code rep to code\n', e)
    if fail_on_error:
      raise e
    program_code = ''
  finally:
    return program_code


def eval_code(code: str):
  test_results = {}
  try:
    context = {}
    exec(code, context)
    test_results = context.get('test_results', {})
  except AssertionError as e:
    test_results['test_failuers'] = test_results.get('test_failuers', 0) + 1
  except Exception as e:
    test_results['code_failure'] = test_results.get('code_failure', 0) + 1

  code_failure = test_results.get('code_failure', 0)
  correct = test_results.get('correct', 0)
  incorrect = test_results.get('incorrect', 0)
  total = (correct + incorrect) or math.inf
  accuracy = (1 - code_failure) * (correct / total)

  results = dict(
    code_failure = code_failure,
    correct = correct,
    incorrect = incorrect,
    accuracy = accuracy,
  )

  return results
  

In [4]:
!ls ~/Downloads

Stanford CS224n 2021 - DL for NLP - Nitzan Barzilay.pdf
Telegram Desktop
config
keywords.tar.gz
ms-recovery-code.pdf
results
~$nl-hw3-solution.docx
תעודת זהות חדשה.pdf


In [5]:
df = pd.read_csv('build/train_complex_utterance_to_code_with_intermediate_40k.csv.gz', compression='gzip')
df = df.sample(n=10, replace=True, random_state=42)

In [6]:
print(df.iloc[2]['code'])

person_reminded = Contact.resolve_from_text("I")
date_time = DateTime.resolve_from_text("10:20 AM")
contents = Content.resolve_many_from_text("the plants need water or walk the dog")
reminders = []
for content in contents:
  reminders = Reminders.find_reminders(person_reminded=person_reminded, date_time=date_time, content=content)
test_reminders = bool(reminders)
Responder.respond(response=test_reminders)
if test_reminders:
  product_names = ProductName.resolve_many_from_text("all hats")
  order = []
  for product_name in product_names:
    order += Shopping.order(product_name=product_name)


In [7]:
results_file_path = "~/Downloads/results/codet5p-220m-text2code-test-82-2023-05-18_202622.csv.gz"
results_df = pd.read_csv(results_file_path, compression='gzip')
results_df.columns

Index(['test_id', 'sample_id', 'sample_minor_id', 'text', 'code', 'test',
       'imports', 'lang_rep', 'code_rep', 'lang_rep_pretty', 'code_rep_pretty',
       'output', 'target'],
      dtype='object')

In [8]:
results_df['sample_id'] = results_df['sample_id'].astype(int)
results_df.set_index(['sample_id', 'sample_minor_id'], inplace=True)
results_df.sort_index(inplace=True)
results_df.head()

test_id   
sample_id sample_minor_id           
0         NaN                   0  \
1         a                   1_a   
          b                   1_b   
2         NaN                   2   
3         a                   3_a   

                                                                        text   
sample_id sample_minor_id                                                      
0         NaN              Check the availability of Pepsi at Walmart and...  \
1         a                If it's raining tomorrow morning, set my alarm...   
          b                If it's raining tomorrow morning, set my alarm...   
2         NaN              Play the new Taylor Swift album and pull up my...   
3         a                      Send a message to dad if it rains tomorrow.   

                                                                        code   
sample_id sample_minor_id                                                      
0         NaN              product_name = ProductName.resolve_from_text("...  \
1         a                date_time = DateTime.resolve_from_text("tomorr...   
          b                date_time = DateTime.resolve_from_text("tomorr...   
2         NaN              album = Album.resolve_from_text("the new Taylo...   
3         a                date_time = DateTime.resolve_from_text("tomorr...   

                                                                        test   
sample_id sample_minor_id                                                      
0         NaN              # test data\ndata_model = DataModel(reset=True...  \
1         a                # test data\ndata_model = DataModel(reset=True...   
          b                # test data\ndata_model = DataModel(reset=True...   
2         NaN              # test data\ndata_model = DataModel(reset=True...   
3         a                # test data\ndata_model = DataModel(reset=True...   

                                                                     imports   
sample_id sample_minor_id                                                      
0         NaN              from entities.generic import *\nfrom entities....  \
1         a                from entities.generic import *\nfrom entities....   
          b                from entities.generic import *\nfrom entities....   
2         NaN              from entities.generic import *\nfrom entities....   
3         a                from entities.generic import *\nfrom entities....   

                                                                    lang_rep   
sample_id sample_minor_id                                                      
0         NaN              [ root [ S [ Command [ Action [ hd [ Check ] ]...  \
1         a                [ root [ S [ Command [ Condition [ If [ Test [...   
          b                [ root [ S [ Command [ Condition [ If [ Test [...   
2         NaN              [ root [ S [ Command [ Action [ hd [ Play ] ] ...   
3         a                [ root [ S [ Command [ Condition [ If [ Body [...   

                                                                    code_rep   
sample_id sample_minor_id                                                      
0         NaN              [ Module [ product_name = ProductName.resolve_...  \
1         a                [ Module [ date_time = DateTime.resolve_from_t...   
          b                [ Module [ date_time = DateTime.resolve_from_t...   
2         NaN              [ Module [ album = Album.resolve_from_text('th...   
3         a                [ Module [ date_time = DateTime.resolve_from_t...   

                                                             lang_rep_pretty   
sample_id sample_minor_id                                                      
0         NaN              [ root [ S [ Command [ Action [ hd [ Check ] ]...  \
1         a                [ root [ S [ Command [ Condition [ If [ Test [...   
          b                [ root [ S [ Command [ Condition [ If [ Test [...   


In [9]:
print(results_df.iloc[0]['text'])
print('\n')
print(results_df.iloc[0]['code'])
print('\n')
print(results_df.iloc[0]['output'])

Check the availability of Pepsi at Walmart and also check it at Walgreens.


product_name = ProductName.resolve_from_text("Pepsi")
location = Location.resolve_from_text("Walmart")
products = Shopping.find_products(product_name=product_name, location=location)
Responder.respond(response=products)

location = Location.resolve_from_text("Walgreens")
products = Shopping.find_products(product_name=product_name, location=location)
Responder.respond(response=products)



product_name = ProductName.resolve_from_text("Pepsi")
product_attribute = ProductAttribute.resolve_from_text("a availability")
location = Location.resolve_from_text("at Walmart")
products = Shopping.find_products(product_name=product_name, product_attribute=product_attribute, location=location)
Responder.respond(response=products)
location = Location.resolve_from_text("at Walgreens")
products = utils.filter(products, location=location)
test_products = bool(products)
Responder.respond(response=test_products)

product_name = Pr

In [10]:
test_code = build_test_code(results_df.iloc[0]['output'], results_df.iloc[0]['imports'], results_df.iloc[0]['test'])
print(test_code)

from entities.generic import *
from entities.calendar import *
from entities.home import *
from entities.map import *
from entities.message import *
from entities.music import *
from entities.navigation import *
from entities.reminder import *
from entities.shopping import *
from entities.weather import *
from actions.calendar import *
from actions.clock import *
from actions.calendar import *
from actions.home import *
from actions.map import *
from actions.messages import *
from actions.music import *
from actions.navigation import *
from actions.reminders import *
from actions.responder import *
from actions.shopping import *
from actions.weather import *
from providers.data_model import DataModel
from datetime import datetime, timedelta
import utils.api_utils as utils
from utils.test_utils import *

# test data
data_model = DataModel(reset=True)
data_product_name_pepsi = ProductName(text="Pepsi")
data_model.append(data_product_name_pepsi)
data_product_name_coca = ProductName(text="

In [11]:
eval_code(test_code)

{'code_failure': 1, 'correct': 0, 'incorrect': 0, 'accuracy': 0.0}

In [14]:
context = {}
exec(test_code, context)
test_results = context.get('test_results', {})

StopIteration: 

In [12]:
from entities.generic import *
from entities.calendar import *
from entities.home import *
from entities.map import *
from entities.message import *
from entities.music import *
from entities.navigation import *
from entities.reminder import *
from entities.shopping import *
from entities.weather import *
from actions.calendar import *
from actions.clock import *
from actions.calendar import *
from actions.home import *
from actions.map import *
from actions.messages import *
from actions.music import *
from actions.navigation import *
from actions.reminders import *
from actions.responder import *
from actions.shopping import *
from actions.weather import *
from providers.data_model import DataModel
from datetime import datetime, timedelta
import utils.api_utils as utils
from utils.test_utils import *

# test data
data_model = DataModel(reset=True)
data_product_name_pepsi = ProductName(text="Pepsi")
data_model.append(data_product_name_pepsi)
data_product_name_coca = ProductName(text="coca cola")
data_model.append(data_product_name_coca)
data_location1 = Location(text="Walmart")
data_model.append(data_location1)
data_location2 = Location(text="Walgreens")
data_model.append(data_location2)
data_location3 = Location(text="CVS")
data_model.append(data_location3)
data_product1 = ProductEntity(
    product_name=data_product_name_pepsi, location=data_location1
)
data_model.append(data_product1)
data_product2 = ProductEntity(
    product_name=data_product_name_pepsi, location=data_location2
)
data_model.append(data_product2)
data_product3 = ProductEntity(
    product_name=data_product_name_coca, location=data_location1
)
data_model.append(data_product3)
data_product4 = ProductEntity(
    product_name=data_product_name_coca, location=data_location2
)
data_model.append(data_product4)
data_product5 = ProductEntity(
    product_name=data_product_name_pepsi, location=data_location3
)
data_model.append(data_product5)
data_product6 = ProductEntity(
    product_name=data_product_name_pepsi, location=data_location1
)
data_model.append(data_product6)

# start code block to test
product_name = ProductName.resolve_from_text("Pepsi")
product_attribute = ProductAttribute.resolve_from_text("a availability")
location = Location.resolve_from_text("at Walmart")
products = Shopping.find_products(product_name=product_name, product_attribute=product_attribute, location=location)
Responder.respond(response=products)
location = Location.resolve_from_text("at Walgreens")
products = utils.filter(products, location=location)
test_products = bool(products)
Responder.respond(response=test_products)

product_name = ProductName.resolve_from_text("Pepsi")
location = Location.resolve_from_text("at Walgreens")
order = Shopping.order(product_name=product_name, location=location)
# end code block to test

# assertions
test_results = {}

iterator = iter(data_model.get_response([ProductEntity]))

expected = [data_product1, data_product6]
actual = next(iterator, None)
response_assertions(expected, actual, test_results)

expected = [data_product2]
actual = next(iterator, None)
response_assertions(expected, actual, test_results)

assert_test(test_results)


In [13]:
test_results

{'correct': 2}